In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import snorkel

%matplotlib inline
from IPython.core.pylabtools import figsize

In [10]:
from snorkel.labeling.apply import PandasLFApplier

POS = 1
NEG = -1
ABSTAIN = 0

In [38]:
#Load in our cleaned data and re-check category labels
media_df = pd.read_csv('/Users/awhite/Documents/snorkel/arabic_news_cleaned.csv')
media_df.category.value_counts()

كرة_القدم                       4301
الأزمة_السورية                 3160
جماعات_مسلحة                    2142
أسواق_النفط                    1364
لاجئون                         1256
رياضات_اخرى                     1100
المعارضة_السورية                1002
صواريخ                           741
التقنية_والمعلومات               639
فضاء                             630
أسلحة_ومعدات_عسكرية             625
الهجرة_إلى_أوروبا              616
الأزمة_الأوكرانية              595
اكتشافات                         577
الأزمة_اليمنية                  566
تفجيرات                          559
مشاهير                           539
البحوث_الطبية                    530
معلومات_عامة                     484
طائرات_حربية                    479
الانتخابات_الأمريكية            447
جرائم                           433
مظاهرات                          422
مؤشرات_اقتصادية                 400
عقوبات_اقتصادية                  397
الاعتراف_بدولة_فلسطين            369
امراض                            323
أولمبياد_ريو_د

There are several broad categories like جماعات_مسلحة (armed groups) and أسواق_النفط (oil markets), then more specific categories such as two for the Syrian conflict: الأزمة_السورية (Syrian conflict) and المعارضة_السورية (Syrian opposition). Let's start with one of the general categories, then move on to Syria.  

In [39]:
#Recoding data for these two classification problems
#Full category names not working, probably because Arabic

media_df = media_df.assign(oil = media_df.category.str.contains("النفط") == True)
media_df = media_df.assign(syria = media_df.category.str.contains("سورية") == True)

media_df.oil = media_df.oil.replace({True:1,False:0})
media_df.syria = media_df.syria.replace({True:1,False:0})

media_df[media_df.oil == 1].head()

,text,category,oil,syria
6208,اسعار تتراجع توقعات انتاج اوبك مستوياته حاليه ...,أسواق_النفط,1,0
6209,تبدا ببنا انابيب سيبيريا روسي اراض قالت شركه غ...,أسواق_النفط,1,0
6210,اسعار ترتفع بدعم ارتفعت اسعار بدعم نزوله تعامل...,أسواق_النفط,1,0
6211,اوبك تجتمع فيينا توقعات ابقا مستويات انتاج تجت...,أسواق_النفط,1,0
6212,روسيا تنوي محافظه مستويات انتاج حاليه طاقه روس...,أسواق_النفط,1,0


In [40]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(media_df, test_size = 0.2, random_state = 0)

train, valid = train_test_split(train, test_size = 0.2, random_state = 0)
train, dev = train_test_split(train, test_size = 0.2, random_state = 0)

Y_train = train["oil"].values
Y_dev = dev["oil"].values
Y_valid = valid["oil"].values
Y_test = test["oil"].values

len(train)

14166

In [42]:
pd.set_option("display.max_colwidth", 0)

dev[dev.oil == 1].sample(5)

,text,category,oil,syria
21304,تماسك اسعار نفط دولارا برميل انخفضت اسعار نفط قليلا تزال متماسكه دولارا برميل توقعات بعودته ارتفاع اجوا ايجابيه لموتمر طاقه عالمي بوتين ندعم تدابير اوبك خفض تجميد انتاج بحلول ساعه بتوقيت موسكو بتوقيت غرينيتش تداول برميل نفط قياس عالمي مزيج برنت سعر دولار منخفضا بنسبه جلسه اغلاق تراجع سعر برميل نفط خام امريكي خفيف بنسبه ليصل دولار اغلقت اسعار جلسه سابقه مرتفعه مستويات قياسيه قرابه عام تفاول مستثمرين متعاملين جرا تاكيد رئيس روسي فلاديمير بوتين كلمته موتمر طاقه عالمي منعقد اسطنبول استعداد بلاده انضمام اتفاق اوبك لتخفيض انتاج تجميده تهيمن سوق ايام موجه تصاعديه خلفيه مباحثات تجر دول اوبك كبار منتجين نفطيين عالم موتمر طاقه عالمي تسريبات ترجح توصل اتفاق تجميد نفط اجتماع اوبك رسمي شهر مقبل فيينا ناديجدا انيوتينا,أسواق_النفط,1,0
6216,اوبك تقرر حفاظ مستوى انتاج مليون برميل يوميا قررت منظمه اوبك اجتما سنوي فيينا محافظه مستويات انتاج سابقه مستوى مليون برميل يوميا لمده اشهر اوبك تعارض انضمام اعضا روسيا اوبك تتطرق لعوده اندونيسيا منظمه سعودي نعيمي اوبك قررت محافظه مستوى انتاج مستوى مليون برميل يوميا معمول قرار اوبك ضغوط فائض اسعار رغبه منظمه اسعار كايران فنزويلا جزائر قررت اوبك اجتما مقبل مقبل اشهر نعيمي اكبر دوله منتجه منظمه منظمه ملتزمين حفاظ مستوى انتاج اعضا منظمه اجتما مساله عوده اندونيسيا عضويه منظمه سعودي انتعاش اسعار باكثر فتره اخيره سجلت ادنى مستوى سنوات بلغت دولارا برميل مسوولون مجتمعون فيينا مبررا لتعديل استراتيجيه انعشت ضعيف استهلاك عالمي كبحت طفره صخري ولايات متحده اوبك معالجه اسئله صعبه اشهر مقبله استعداد اعضا ايران ليبيا لزياده انتا نفطي سنوات تراجع تسعى ايران اعضا منظمه لافساح مجال زياده معروض مليون برميل يوميا تخفيف عقوبات غربيه مفروضه توصل لاتفاق مجموعه بشان برنام نووي تامل ليبيا تعاني صراع مسلح زياده انتاج حوالي مليون برميل يوميا اعاده تشغيل موانئ رئيسيه,أسواق_النفط,1,0
6208,اسعار تتراجع توقعات انتاج اوبك مستوياته حاليه هبطت اسعار بفعل صعود دولار توقعات تبقي منظمه اوبك انتا مرتفعا مخاوف استمرار معروض اسواق اظهرت نتائج انتاج منظمه اوبك اعلى مستوى عامين مليون برميل يوميا ماضي تجتمع منظمه اوبك فيينا مقبل يتوقع تغير سياس انتاجيه سيقود لاستمرار تخمه معروض اسواق عالميه مستقبل منظور بحلول بتوقيت موسكو انخفضت عقود آجله لسعر مزيج برنت تسليم بنسبه يعادل دولار دولار برميل تراجعت عقود آجله امريكي تسليم بنسبه يعادل سنتا دولارا برميل ارتفع موشر دولار بنسبه مقابل عملات يجعل تكلفه حائزي عملات يشار اسعار قفزت بنحو ماضي اكبر ارتفاع خلفيه تراجع عمليات تنقيب ولايات متحده,أسواق_النفط,1,0
21288,جزائر تكشف تفاصيل مقترح سعودي لاستقرار سوق نفط تعول جزائر يتمخض اجتماع دول منتجه نفط مرتقب امرين تثبيت انتاج نفط تقليصه سيحقق استقرار سوق نفط تعاني تخمه معروض يعقد اعضا منظمه اوبك الـ اجتماعا رسمي جزائر فتره ستحضر روسيا عضو منظمه اجتماع ستبحث دول منتجه اوضاع سوق نفط اكد زير طاقه جزائري نور دين بوطرفه عشيه منتدى هدف اساسي اجتماع تحقيق استقرار اسعار نفط خاصه استمر انتاج نفط عالمي مليون برميل يوميا اسعار ستبقى مستويات متدنيه لعام سينعكس استثمارات قطاع نفط اقتصادات دول منتجه خاصه اقتصاد عالمي عامه انتاج دول منظمه اوبك مليون برميل نفط براي وزير جزائري سعر برميل نفط دولارا سعر عادل سيساهم استقرار اقتصادات دول منتجه خام بوطرفه سعوديه اكبر منتجي نفط منظمه اوبك اقترحت خفض حجم انتاج نفط الف برميل يوميا يمثل نحو انتاج مملكه اَب مليون برميل يوميا اضاف بوطرفه دول اوبك ستبحث مقترح اجتماع جزائر موكدا ترحيب بلاده مقترح شهد انتاج اوبك ارتفاعا ملحوظا عام ارتفع انتاج نحو مليون برميل يوميا سجل منظمه مليون برميل يوميا بلغ اَب ماضي وقت تعاني سوق نفط تخمه مفرطه انعكس اسعار تطور انتاج منظمه دول نفط اوبك مليون برميل نفط جهته قلل زير طاقه روسي كسندر نوفاك تبعات فشل اجتماع جزائر روسيا خاصه اجتماعا مماثلا انهار توتر علاقات سعوديه ايران وزير روسي عدم توصل اتفاق اجتماع يكون كارثيا نسبه لروسيا ابرامه سيعمل تسريع عمليه استقرار سوق نفط جهود مبذوله حاليا مفاوضات منتجي نفط مرجح تسفر تحديات فشل جهود دعم اسعار خام اشار تقرير صادر بنك اوف امريكا توتر سياسي سعوديه ايران امكانيه عوده امدادات نفطيه شهدت توقفا فتره خاصه ليبيا نيجيريا يضيفان صعوبه لامكانيه توصل لاتفاق منتجين رجح تقرير تُبقي تحديات فائض معروض سوق مستمرا عام,أسواق_النفط,1,0
6348,يبدا تعاملات اسبوع مرتفعا ارتفعت اسعار قالت اوبك تتوقع عالمي قويا قادم تاثرت ببيانات ضعيفه تجاره صينيه دعما تراجع دولار يجعل

In [47]:
#testing regex quirks in Arabic text
test = "الجزائر"
test2 = "الجزاىر"

test.count(r"الجزا*ر")

#Wildcard not working

0

In [49]:
oil = r"برميل|نفظ|بترول"

@labeling_function()
def oil_terms(x):
    return POS if re.search(oil, x.text) else ABSTAIN

In [50]:
econ = r"سعر|اسعار|ستثمار|اقتصاد|سوق|اسواق"

@labeling_function()
def econ_terms(x):
    return POS if re.search(econ, x.text) else ABSTAIN

In [60]:
conflict = r"ضبرات|معارك|سلح"

@labeling_function()
def conflict_terms(x):
    return NEG if re.search(conflict, x.text) else ABSTAIN

In [54]:
exclude = r"كرة القدم|فلم|افلام"

@labeling_function()
def misc_exclude(x):
    return NEG if re.search(exclude, x.text) else ABSTAIN

In [63]:
countries = r"خليجي|الجزائر|ليبي|نيجيري|سعودي|روسي|امريك"

@labeling_function()
def oil_countries(x):
    return POS if re.search(countries, x.text) else ABSTAIN


In [64]:
lfs = [oil_terms, econ_terms, conflict_terms, misc_exclude, oil_countries]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=train)
L_dev = applier.apply(df=dev)

100%|██████████| 3542/3542 [00:01<00:00, 3170.85it/s]


In [65]:
from snorkel.labeling.analysis import LFAnalysis

LFAnalysis(L=L_dev, lfs=lfs).lf_summary(Y=Y_dev)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
oil_terms,0,"[0, 1]",1.000000,1.000000,0.618577,104,0,0.974026
econ_terms,1,"[0, 1]",1.000000,1.000000,0.618577,143,0,0.897516
conflict_terms,2,[0],0.792772,0.792772,0.484190,2640,168,0.940171
misc_exclude,3,[0],0.992660,0.992660,0.614060,3339,177,0.949659
oil_countries,4,"[0, 1]",1.000000,1.000000,0.618577,151,0,0.458780


In [85]:
from snorkel.analysis.error_analysis import get_label_buckets

buckets = get_label_buckets(Y_dev, L_dev[:, 1])
dev.iloc[buckets[(POS, ABSTAIN)]].sample(5, random_state=1)

,text,category,oil,syria
21065,غارديان تتحصل ثائق داعش لبنا دوله خلافه نشرت صحيفه غارديان بريطانيه ثيقه سريه مسربه تعود لداعش تكشف تفاصيل خطط تنظيم ارهابي لبنا اسس يدعي دوله خلافه مرحله اولى اعلان تنظيم خلافه اسلاميه اراضي عراق سوريا ركز قادته اصدار اوامر تخص جوانب سطحيه حياه اجتماعيه حدود دوله مزعومه حظر ارتدا ملابس اسلاميه تربيه حمام اعتبار اخيره مضيعه وقت انتقل داعش اصدار لوائح تعليمات تخص دوله بحد نشرت غارديان ثيقه سريه مكونه صفحه تحمل عنوان مبادئ اداره دوله اسلاميه تضم وثيقه مذكوره مبادئ نظام تربوي اعلامي لداعش آليات دعايته علاقاته خارجيه تحكم تجاره نفط غاز تصور وثيقه نظاما شديد تعقيد لتسيير معسكرات تنقسم اعداديه خاصه اطفال تدريبيه لمسلحين محترفين تشير صحيفه برنامج يبرز مدى جديه تعاطي تنظيم ارهابي قضيه اسس دوله غارديان تحدي يواجهه ائتلاف غربي داعش ينحصر تنفيذ مهمه قتاليه عاديه تنظيم اعتباره مجرد جمله مسلحين تنقل صحيفه ستينلي ماكريستال جنرالات احتياط امريكيين قوله غرب بتعامله داعش عصابه مجانين عاديين يجازف وقوع خطر تقدير قواه صحيفه بريطانيه تشير ثائق داعش صادره مدار اشهر خمسه اخيره تركز اجراات امن تعبئه تجنيد امر يدل تنامي اعراض بارانويا صفوف تنظيم فرض حظرا شبكات الـ خاصه اعلن عاما هاربين ساحات قتال نقص جنود تنظيم نوفوستي,أسواق_النفط,1,0
21414,مصر ثالث كشف غاز لشركه اعلنت شركه نفط بريطانيه اكتشافا جديدا غاز طبيعي منطقه امتياز شمال دمياط بحريه بشرق دلتا نيل يوكد جود مكامن لغاز منطقه مصريه يعد ثالث نجاح لنشاطـ قطاع كشفي سلامات اتول وتم حفر بئر استكشافيه اطلق اسم قطاميه ضحله بعمق اجمالى مترا مياه امتار تقريبا تدل معطيات اوليه جود طبقه حامله غاز بسماكه مترا تجري حاليا دراسه خيارات متعلقه بربط كشف بنيه تحتيه قائمه بوب دودلي رئيس تنفيذى لمجموعه عالميه يعتبر بئر قطاميه كشف منطقه امتياز نقوم فعل بتطوير حقل اتول تقييم اكتشاف سلامات نجاحنا مستمر مجال يوكد ايماننا دلتا نيل حوض غاز طراز عالمي تقع بئر قطاميه ضحله شمال مدينه دمياط تبعد جنوب غرب حقل سلامات غرب تسهيلات حقل حابى بحريه يذكر شركه تنتج نحو اجمالي غاز مصري فريد غايرلي,أسواق_النفط,1,0
21057,موسكو تعلق توريدات روسي اوكرانيا اعلن يكسي ميللر رئيس شركه غازبروم روسيه تعليق امدادات طبيعي روسي اوكرانيا استلام دفعات نقديه جديده كييف توريدات روسيا تعتزم ايقاف ارداتها لاوكرانيا تنظر تعليق امدادات رئيس شركه غازبروم روسيه شركته تتلق دفعات جديه شركه طاقه اوكرانيه نفطوغاز توريدات جديده تعليق امدادات بحلول ساعه بتوقيت موسكو روسيا اوكرانيا مفوضيه اوروبيه نهايه ماضي بروتوكول مبدئي بشان توريدات روسي اوكرانيا يعرف حزمه شتويه اتفاق قيام كييف بدفع مسبقا روسي مورد تضمن اتفاق كييف تخفيضات توفير تمويل لازم لاوكرانيا مفوضيه اوروبيه لشرا كميات مطلوبه طبيعي لازم مستودعات اوكرانيه بهدف ضمان امدادات مستقره روسي اوروبا اراضي اوكرانيه,أسواق_النفط,1,0
21133,شركه بريطانيه تسحب موظف جزائر كشفت شركه بريتش بتروليوم ستسحب موظف محطتي عين صالح ميناس غاز جزائر اسبوعين مقبلين اعلنت شركه نفط غاز نرويجيه شتات اويل تشغل منشاتين شراكه خطط لتخفيض موظف هجوم قذائف صاروخيه منشاه عين صالح يسفر قوع اصابات خسائر قالت شركه بريطانيه بيان قررت اجرا نقل موقت مراحل لجميع افراد طاق مشروعي عين صالح اميناس جزائر اسبوعين مقبلين اتخذ قرار كاجرا احترازي وقت ماضي استهدف مسلحون مجهولون قاعده نفطيه بعين صالح جزائر صواريخ انبا قوع ضحايا نقلت صحيفه خبر جزائريه مصادر امنيه عناصر جيش وطني شعبي تصدت لمحاوله اعتدا ارهابي منشاه نفطيه اقعه منطقه خريشبه كلم عين صالح اضافت جماعه ارهابيه استهدفت قاعده نفطيه تابعه شركه بريطانيه بريتيش بتروليوم قذائف تمهيدا استيلا تدخل عناصر قوات جيش وطني شعبي احبط محاوله,أسواق_النفط,1,0
6757,تشغيل آبار طبيعي جديده متوسط كشفت زاره بترول مصريه نجاح ايني ايطاليه بريتش بتروليوم بتشغيل آبار جديده طبيعي متوسط اكدت وزاره مليون مكعب يوميا انتاج بنهايه ماضي بفضل تمكن شركات اجنبيه مذكوره تشغيل آبار اربعه متوسط دلتا صحرا غربيه افاد بيان وزاره آبار اربعه بمنطقه شمال دلتا تابع لشركه بمعدل انتاج مليون مكعب يوميا بمعدل انتاج مليون مكعب يوميا علما بئرين تابعين لحقل نورس تشرف ساهم بزياده انتاج ليصل مليون مكعب يوميا بمعدل انتاج مليون مكعب يوميا رابع سترا صحرا غربيه بمعدل انتاج مليون مكعب يوميا يذكر متوسط انتاج يبلغ حوالي مليار مكعب يوميا تستهلك بلاد حوالي مليار مكعب تعوض استيراد تراجع انتاج زياده استهلاك ايام دوله مصدره طاقه تحويل امدادات طاقه محليه مستورده طاقه باستيراد طبيعي مسال,أسواق_النفط,1,0


Hm...the first entry here definitely isn't about oil market. Bad labeled data! What an unusual problem!

Other errors seem to indicate we can just expand the keywords in existing LFs and get better results.

In [ ]:
oil = r"برميل|نفظ|بترول"
econ = r"سعر|اسعار|ستثمار|اقتصاد|سوق|اسواق"
conflict = r"ضبرات|معارك|سلح"
exclude = r"كرة القدم|فلم|افلام"
countries = r"خليجي|الجزائر|ليبي|نيجيري|سعودي|روسي|امريك"